In [1]:
!nvidia-smi

Wed Jun  9 04:12:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow_datasets as tfds 
import tensorflow_hub as hub

In [3]:
(train_data, test_data),data_info = tfds.load(name = 'food101',
                                              split = ['train', 'validation'],
                                              as_supervised = True,
                                              batch_size = 32,
                                              with_info = True)

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteXPXV4R/food101-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteXPXV4R/food101-validation.tfrecord


Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [4]:
class_names = data_info.features['label'].names

In [23]:
for image,label in train_data.take(1):
  print(image,label)

tf.Tensor(
[[[[233 253 251]
   [231 253 250]
   [228 254 251]
   ...
   [ 85  82  89]
   [ 68  67  75]
   [ 57  57  67]]

  [[232 254 252]
   [229 254 251]
   [226 255 251]
   ...
   [121 116 120]
   [100  99 104]
   [ 86  85  91]]

  [[228 254 253]
   [226 255 253]
   [223 255 252]
   ...
   [164 159 155]
   [145 141 138]
   [128 127 125]]

  ...

  [[ 66 112 164]
   [ 67 113 163]
   [ 55  99 148]
   ...
   [  5  14  23]
   [  9  18  27]
   [  8  17  26]]

  [[ 76 123 177]
   [ 75 122 176]
   [ 70 116 168]
   ...
   [  5  14  23]
   [  9  18  25]
   [  7  16  23]]

  [[ 80 129 185]
   [ 71 121 174]
   [ 74 121 175]
   ...
   [  7  16  25]
   [ 11  20  27]
   [ 10  19  26]]]


 [[[ 23  25  22]
   [ 13  15  12]
   [  7   9   6]
   ...
   [ 75  70  51]
   [ 77  74  55]
   [ 79  76  57]]

  [[ 22  24  21]
   [ 14  16  13]
   [  9  11   8]
   ...
   [ 77  72  53]
   [ 77  74  55]
   [ 78  75  56]]

  [[ 21  23  20]
   [ 17  19  16]
   [ 12  14  11]
   ...
   [ 78  71  53]
   [ 78  73  54]


In [24]:
len(class_names)

101

In [25]:
base_model = tf.keras.applications.EfficientNetB4(include_top= False)
base_model.trainable = False
inputs = tf.keras.layers.Input(shape =(224,224,3))
x = base_model(inputs,training = False)
x = tf.keras.layers.GlobalAveragePooling2D(name = 'pooling_layer')(x)
x = tf.keras.layers.Dense(len(class_names))(x)
outputs = tf.keras.layers.Activation('softmax',dtype = tf.float32,name = 'softmax_float32')(x)
model = tf.keras.Model(inputs,outputs)

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(),metrics = ['accuracy'])

In [27]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb4 (Functional)  (None, None, None, 1792)  17673823  
_________________________________________________________________
pooling_layer (GlobalAverage (None, 1792)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 101)               181093    
_________________________________________________________________
softmax_float32 (Activation) (None, 101)               0         
Total params: 17,854,916
Trainable params: 181,093
Non-trainable params: 17,673,823
_________________________________________________________________


In [28]:
!nvidia-smi

Wed Jun  9 04:39:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    34W / 250W |   8715MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
# tf.keras.mixed_precision.set_global_policy(policy = 'mixed_float16')

In [32]:
history_b4 = model.fit(train_data, 
                       steps_per_epoch = len(train_data),
                       validation_data = test_data, 
                       validation_steps = len(test_data),
                       epochs = 100,
                       callbacks = [tf.keras.callbacks.TensorBoard('/logs/effi_b4'),
                                    tf.keras.callbacks.ModelCheckpoint('/checkpoints/effi_b4',save_best_only = True,save_weights_only= True,monitor = 'val_loss'),
                                    tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 3 )]
                       )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/100
2368/2368 [==============================] - 1295s 545ms/step - loss: 1.5727 - accuracy: 0.6106 - val_loss: 1.0257 - val_accuracy: 0.7269
Epoch 2/100
2368/2368 [==============================] - 1286s 543ms/step - loss: 1.0927 - accuracy: 0.7138 - val_loss: 0.9147 - val_accuracy: 0.7528
Epoch 3/100
2368/2368 [==============================] - 1285s 543ms/step - loss: 0.9610 - accuracy: 0.7465 - val_loss: 0.8662 - val_accuracy: 0.7638
Epoch 4/100
2368/2368 [==============================] - 1286s 543ms/step - loss: 0.8749 - accuracy: 0.7694 - val_loss: 0.8392 - val_accuracy: 0.7684
Epoch 5/100
2368/2368 [==============================] - 1285s 543ms/step - loss: 0.8102 - accuracy: 0.7867 - val_loss: 0.8228 - val_accuracy: 0.7727
Epoch 6/100
2368/2368 [==============================] - 1285s 543ms/step - loss: 0.7579 - accuracy: 0.8006 - val_loss: 0.8126 - val_accuracy: 0.7744
Epoch 7/100
2368/2368 [==============================] - 1286s 543ms/step - loss: 0.7140 - accuracy:

KeyboardInterrupt: ignored

In [35]:
!tensorboard dev upload --logdir ./logs/effi_b4/train --one_shot

2021-06-09 07:53:17.112641: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/6xt5Jh8ESlGoPNk4rl1DpA/

[2021-06-09T07:53:18] Started scanning logdir.
[2021-06-09T07:53:18] Done scanning logdir.
TensorBoard was run in `one_shot` mode, but did not find any uploadable data in the specified logdir: ./logs/effi_b4/train
An empty experiment was created. To delete the empty experiment you can execute the following

    tensorboard dev delete --experiment_id=6xt5Jh8ESlGoPNk4rl1DpA


Done.


In [38]:
model.predict(test_data)

array([[2.3329249e-03, 4.1104605e-07, 6.9773450e-06, ..., 1.6494880e-04,
        4.7078873e-05, 1.1873422e-02],
       [2.5941914e-05, 6.8557488e-06, 1.4896577e-05, ..., 3.2451971e-06,
        2.9608886e-05, 8.1142971e-06],
       [5.2227997e-06, 1.7796892e-05, 6.1857384e-07, ..., 7.8594258e-06,
        2.2219772e-06, 1.6381520e-06],
       ...,
       [1.0486386e-02, 4.9370014e-05, 1.4336074e-04, ..., 8.3115045e-03,
        4.9388142e-05, 9.8782446e-05],
       [7.2637005e-05, 8.2372662e-07, 1.0944637e-07, ..., 1.4927537e-05,
        3.6923484e-06, 8.6437110e-05],
       [1.3837371e-03, 4.3501805e-07, 1.2590991e-04, ..., 1.5426290e-02,
        1.8641751e-05, 1.2683995e-04]], dtype=float32)

In [39]:
model.save('high_acc.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [40]:
model.evaluate(train_data)

2368/2368 [==============================] - 967s 408ms/step - loss: 0.5838 - accuracy: 0.8458


[0.5837501883506775, 0.8458217978477478]

In [41]:
model.evaluate(test_data)

790/790 [==============================] - 322s 407ms/step - loss: 0.8008 - accuracy: 0.7774


[0.800758421421051, 0.7773861289024353]